# Module 10: Automation for Data & ML (Expanded)

**Learning Objectives:**
- Schedule Python scripts using cron (Linux/Mac) and Task Scheduler (Windows)
- Implement structured logging and error alerting
- Organize data/ML projects with best-practice folder structures
- Use n8n to orchestrate automated ML pipelines, dataset refreshes, and report delivery

> **Why does this matter?** Running a notebook manually is fine for exploration. A production data system runs *automatically*, recovers from errors, and sends you an alert if something goes wrong.

In [ ]:
import os
import sys
import json
import time
import logging
import smtplib
import textwrap
import subprocess
import traceback
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from email.message import EmailMessage
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded!")

---
## 1. Folder Structure Best Practices

A well-organized project is the foundation of automation. Using a standard structure helps collaborators understand the project and allows automated scripts to find files reliably.

In [ ]:
# Canonical folder structure for an analytics/ML project
PROJECT_STRUCTURE = """
my_project/
├── data/
│   ├── raw/             # Original, unmodified source files
│   ├── processed/       # Cleaned, feature-engineered data
│   └── exports/         # Final outputs (reports, predictions)
├── notebooks/           # Exploratory Jupyter notebooks
├── src/
│   ├── __init__.py
│   ├── ingest.py        # Data ingestion functions
│   ├── clean.py         # Cleaning & validation functions
│   ├── features.py      # Feature engineering
│   ├── model.py         # Training, predict, evaluate
│   └── report.py        # Report generation
├── scripts/
│   ├── run_pipeline.py  # Full pipeline entry point
│   └── refresh_data.py  # Scheduled data refresh
├── logs/                # Rotating log files
├── config/
│   └── config.json      # Environment-agnostic settings
├── models/              # Saved ML model artifacts
├── tests/               # Unit tests
├── requirements.txt
└── README.md
"""
print(PROJECT_STRUCTURE)

In [ ]:
# Scaffold the structure programmatically
def scaffold_project(base_path, name='my_project'):
    root = Path(base_path) / name
    dirs = [
        'data/raw', 'data/processed', 'data/exports',
        'notebooks', 'src', 'scripts', 'logs', 'config', 'models', 'tests'
    ]
    for d in dirs:
        (root / d).mkdir(parents=True, exist_ok=True)

    # Init files
    (root / 'src' / '__init__.py').touch()
    (root / 'config' / 'config.json').write_text(json.dumps(
        {'env': 'development', 'data_path': 'data/raw', 'log_level': 'INFO'}, indent=2
    ))
    (root / 'README.md').write_text(f"# {name}\n\nProject description here.\n")
    (root / 'requirements.txt').write_text("numpy\npandas\nscikit-learn\nmatplotlib\nseaborn\nrequests\n")

    print(f"✅ Project scaffolded at: {root.resolve()}")
    for p in sorted(root.rglob('*')):
        indent = '  ' * (len(p.relative_to(root).parts) - 1)
        print(f"{indent}{'📁' if p.is_dir() else '📄'} {p.name}")
    return root

project_root = scaffold_project('.', 'demo_project')

---
## 2. Structured Logging

Using Python's `logging` module instead of `print()` gives you:
- Severity levels (DEBUG, INFO, WARNING, ERROR, CRITICAL)
- Timestamps and source location
- File rotation and multiple handlers

In [ ]:
from logging.handlers import RotatingFileHandler

def setup_logger(name, log_file='logs/pipeline.log', level=logging.INFO):
    """Create a logger with console + rotating file handler."""
    os.makedirs('logs', exist_ok=True)
    
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.handlers = []  # Clear existing handlers

    fmt = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)-8s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

    # Console handler
    ch = logging.StreamHandler()
    ch.setFormatter(fmt)
    logger.addHandler(ch)

    # Rotating file handler (10 MB max, 5 backups)
    fh = RotatingFileHandler(log_file, maxBytes=10*1024*1024, backupCount=5)
    fh.setFormatter(fmt)
    logger.addHandler(fh)

    return logger

logger = setup_logger('data_pipeline')
logger.info('Pipeline logger initialized.')
logger.warning('This is a warning example.')
logger.error('This is a simulated error entry.')

In [ ]:
# Decorator: auto-log function calls with timing
import functools

def log_step(logger):
    """Decorator that logs a pipeline step's start, duration, and errors."""
    def decorator(fn):
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            logger.info(f"▶ Starting: {fn.__name__}")
            start = time.time()
            try:
                result = fn(*args, **kwargs)
                elapsed = time.time() - start
                logger.info(f"✅ Completed: {fn.__name__} in {elapsed:.3f}s")
                return result
            except Exception as e:
                logger.error(f"❌ Failed: {fn.__name__} — {e}")
                logger.debug(traceback.format_exc())
                raise
        return wrapper
    return decorator

@log_step(logger)
def load_raw_data(n=1000):
    time.sleep(0.1)  # Simulate I/O
    return pd.DataFrame({'x': np.random.randn(n), 'y': np.random.randn(n)})

@log_step(logger)
def clean_data(df):
    return df.dropna()

df = load_raw_data()
df_clean = clean_data(df)

---
## 3. Error Handling & Alerts

Production pipelines must gracefully handle failures and optionally notify humans.

In [ ]:
# Generic pipeline error handler
def safe_run(fn, *args, on_error=None, **kwargs):
    """
    Wraps a function call. On exception, calls on_error(e) if provided.
    Returns (result, error) tuple.
    """
    try:
        return fn(*args, **kwargs), None
    except Exception as e:
        if on_error:
            on_error(e)
        return None, e

def send_slack_alert(webhook_url, message):
    """Send error alert to a Slack webhook."""
    payload = {'text': f":rotating_light: *Pipeline Alert*\n{message}"}
    try:
        r = requests.post(webhook_url, json=payload, timeout=5)
        r.raise_for_status()
        print("Slack alert sent.")
    except Exception as e:
        logger.warning(f"Could not send Slack alert: {e}")

# Example alert function (replace with your Slack webhook)
SLACK_WEBHOOK = 'https://hooks.slack.com/YOUR_WEBHOOK_URL'

def on_pipeline_error(e):
    msg = f"Error in data pipeline: `{type(e).__name__}: {e}`"
    logger.critical(msg)
    # send_slack_alert(SLACK_WEBHOOK, msg)  # Uncomment when webhook is configured
    print(f"[ALERT TRIGGERED] {msg}")

# Test safe_run
def risky_step():
    raise ValueError("Source file missing from data/raw/!")

result, err = safe_run(risky_step, on_error=on_pipeline_error)
print(f"Result: {result} | Error caught: {err}")

---
## 4. Scheduling: Cron (Linux/Mac) & Task Scheduler (Windows)

### Linux/Mac — cron job
```bash
# Open crontab editor
crontab -e

# Run python script every day at 6:00 AM
0 6 * * * /path/to/venv/bin/python /path/to/scripts/run_pipeline.py >> /path/to/logs/cron.log 2>&1

# Run every Monday at 8:00 AM
0 8 * * 1 /path/to/venv/bin/python /path/to/scripts/refresh_data.py
```

### Windows — Task Scheduler (via PowerShell)
```powershell
$action = New-ScheduledTaskAction -Execute 'python.exe' `
    -Argument 'C:\projects\scripts\run_pipeline.py' `
    -WorkingDirectory 'C:\projects'

$trigger = New-ScheduledTaskTrigger -Daily -At '6:00AM'

Register-ScheduledTask -Action $action -Trigger $trigger `
    -TaskName 'DataPipeline' -Description 'Daily ML Pipeline'
```

In [ ]:
# Portable scheduler using Python (no OS setup needed)
# Install: pip install schedule
"""
import schedule
import time

def daily_pipeline():
    print(f'[{datetime.now()}] Running daily pipeline...')
    # ... your pipeline code here

schedule.every().day.at('06:00').do(daily_pipeline)
schedule.every().monday.at('08:00').do(daily_pipeline)

while True:
    schedule.run_pending()
    time.sleep(60)  # Check every minute
"""
print("(Code shown for reference — run as a standalone script, not in a notebook)")

---
## 5. n8n for Automation

**n8n** is an open-source workflow automation tool (like Zapier, but self-hosted). It lets you visually wire up triggers and actions — no code required for the glue.

### Key n8n Concepts
| Term | Meaning |
|---|---|
| **Workflow** | A visual sequence of nodes |
| **Trigger Node** | What starts the workflow (schedule, webhook, email) |
| **Action Node** | What happens (run Python, send email, call API) |
| **Execute Command** | Node that runs a shell/Python command |
| **Webhook Node** | HTTP endpoint that triggers a workflow |
| **HTTP Request Node** | Calls any external REST API |

### 5.1  Auto-Run Python Scripts via n8n

In n8n, create a workflow:
```
[Schedule Trigger: Daily 06:00] → [Execute Command]
```
In the **Execute Command** node:
```bash
cd /home/user/projects && source venv/bin/activate && python scripts/run_pipeline.py
```
The output of the script (stdout/stderr) is available in subsequent nodes.

### 5.2  Auto-Refresh Datasets

Workflow for refreshing a dataset from an API and saving to Parquet:
```
[Schedule Trigger: Every 1 Hour]
  → [HTTP Request: GET https://api.example.com/data]
  → [Execute Command: python -c "import pandas as pd; ..."]
  → [Write Binary File: data/processed/latest.parquet]
```

### 5.3  Trigger ML Prediction Workflow

Expose your ML model as a webhook endpoint (n8n receives data, runs the model, returns a prediction):
```
[Webhook: POST /predict]
  → [Execute Command: python scripts/predict.py --input='{{ $json.data }}']
  → [Respond to Webhook: {{ $json.prediction }}]
```

In [ ]:
# Example: predict.py script triggered by n8n webhook
predict_script = textwrap.dedent("""
#!/usr/bin/env python
# scripts/predict.py
# Called by n8n: python predict.py --input='{"age":35,"years_experience":10,"score":85}'

import sys
import json
import argparse
import joblib
import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument('--input', type=str, required=True)
args = parser.parse_args()

input_data = json.loads(args.input)
model = joblib.load('models/salary_model.pkl')
scaler = joblib.load('models/scaler.pkl')

features = np.array([[input_data['age'], input_data['years_experience'], input_data['score']]])
features_scaled = scaler.transform(features)
prediction = model.predict(features_scaled)[0]

print(json.dumps({'prediction': round(prediction, 2)}))
""")
print(predict_script)

### 5.4  Send Automated PDF/HTML Reports to Email / Teams / Slack

The three-step pattern:
1. Python generates a report (HTML or PDF)
2. n8n reads the file
3. n8n sends it via Email / Microsoft Teams / Slack node

In [ ]:
# Step 1: Generate an HTML report from Python
def generate_html_report(df, output_path='data/exports/report.html', title='Daily Report'):
    """Generate a styled HTML report from a DataFrame."""
    now = datetime.now().strftime('%Y-%m-%d %H:%M')
    stats_html = df.describe().round(2).to_html(classes='table', border=0)
    table_html = df.head(20).to_html(index=False, classes='table', border=0)

    html = f"""<!DOCTYPE html>
<html><head>
<meta charset='utf-8'>
<style>
  body {{ font-family: Arial, sans-serif; margin: 32px; }}
  h1 {{ color: #2c3e50; }}
  .meta {{ color: gray; font-size: 0.9em; }}
  .table {{ border-collapse: collapse; width: 100%; margin-bottom: 24px; }}
  .table th {{ background: #2980b9; color: white; padding: 8px; text-align: left; }}
  .table td {{ padding: 6px 8px; border-bottom: 1px solid #eee; }}
  .table tr:hover {{ background: #f5f5f5; }}
</style>
</head><body>
<h1>{title}</h1>
<p class='meta'>Generated: {now}</p>
<h2>Summary Statistics</h2>{stats_html}
<h2>Data Preview (first 20 rows)</h2>{table_html}
</body></html>"""

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        f.write(html)
    print(f"✅ Report written to: {output_path}")
    return output_path

sample_df = pd.read_csv('data/transactions_large.csv', nrows=500) if os.path.exists('data/transactions_large.csv') else pd.DataFrame({'a': range(20), 'b': np.random.randn(20)})
report_path = generate_html_report(sample_df, title='Daily Transaction Report')

In [ ]:
# n8n Workflow for report delivery (conceptual)
n8n_workflow_json = {
    "name": "Daily Report Pipeline",
    "nodes": [
        {"name": "Schedule Trigger", "type": "n8n-nodes-base.scheduleTrigger", "parameters": {"rule": {"interval": [{"field": "hours", "hoursInterval": 24}]}}},
        {"name": "Run Python Report", "type": "n8n-nodes-base.executeCommand", "parameters": {"command": "python scripts/generate_report.py"}},
        {"name": "Read Report File", "type": "n8n-nodes-base.readBinaryFile", "parameters": {"filePath": "data/exports/report.html"}},
        {"name": "Send to Teams", "type": "n8n-nodes-base.microsoftTeams", "parameters": {"operation": "sendMessage", "messageType": "text", "message": "Daily report attached."}},
        {"name": "Send to Slack", "type": "n8n-nodes-base.slack", "parameters": {"operation": "postMessage", "channel": "#data-reports", "text": "📊 Daily Report ready!"}}
    ]
}
print("n8n workflow structure (for reference):")
print(json.dumps(n8n_workflow_json, indent=2))

---
## 6. Exercises

1. Write a full `run_pipeline.py` script using the `log_step` decorator to chain: `load → clean → aggregate → save_parquet`.
2. Set up a cron job (or Windows Task) that runs your pipeline every day at 7 AM.
3. Extend `generate_html_report` to include a bar chart image (save with matplotlib and embed as base64).
4. Design an n8n workflow that: (a) fetches weather data every 6 hours, (b) appends it to a Parquet file, (c) sends a Slack message if temperature exceeds 35°C.
5. Write a `send_teams_alert(webhook_url, message)` function using a Microsoft Teams Incoming Webhook.